# Exercices SQL

Récupérer les données dans 'produits csv'

Les insérer dans une nouvelle base de deonnées qu’on aura conçue
Faire ça en python



In [2]:
import numpy as np
import pandas as pd

prod_df = pd.read_csv('./Produits - Feuille 1.csv')

print(prod_df.shape[0])

fam_df = prod_df.groupby(['FAMILLE ARTICLE']).count().drop(['LIBELLE ARTICLE', 'CONDITION', 'PU HT'], axis=1)
cond_df = prod_df.groupby(['CONDITION']).count().drop(['LIBELLE ARTICLE', 'FAMILLE ARTICLE', 'PU HT'], axis=1)

1863


Une fois qu’on a ces infos, on sait qu’on devra créer une table pour chacunes de ces informations en y associant un identifiant

Se connecter avec PyMysql

In [3]:
import mysql.connector
import random

db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'sakila'
)

cur = db.cursor()

cur.execute("SELECT * FROM actor LIMIT 1")
res = cur.fetchall()
print(res)


[(1, 'PENELOPE', 'GUINESS', datetime.datetime(2006, 2, 15, 4, 34, 33))]


Créons la Base de Données sur mon server

In [4]:
#create random id
actor_ids = list(range(1,201))
random.shuffle(actor_ids)
print(actor_ids[0])

sql = '''
SELECT f.title, a.first_name, a.last_name, a.actor_id
FROM film_actor AS fa
JOIN film AS f
ON fa.film_id = f.film_id
JOIN actor AS a
ON fa.actor_id = a.actor_id
WHERE fa.actor_id = %s;
'''

cur.execute(sql, (actor_ids[0],))
res = cur.fetchall()
print(len(res))


114
25


Essayons de créer une table sur ma base perso «produ1»

In [5]:
db = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'produ1'
)

cur = db.cursor()

cur.execute("DROP TABLE produit")
cur.execute("DROP TABLE famille")
cur.execute("DROP TABLE condit")

req = ['''
CREATE TABLE IF NOT EXISTS produit (
    prod_id INT NOT NULL AUTO_INCREMENT,
    fam_id INT,
    cond_id INT,
    code_art INT,
    libelle VARCHAR(80) NOT NULL UNIQUE,
    prix DECIMAL(6,2) NOT NULL,
    PRIMARY KEY (prod_id)
)''','''
CREATE TABLE IF NOT EXISTS famille (
    fam_id INT,
    nom VARCHAR(80) NOT NULL UNIQUE,
    PRIMARY KEY (fam_id)
)''','''
CREATE TABLE IF NOT EXISTS condit (
    cond_id INT,
    nom VARCHAR(80) NOT NULL UNIQUE,
    PRIMARY KEY (cond_id)
)
''']

for r in req : cur.execute(r)

cur.execute("SHOW TABLES")

for x in cur: print(x)

('condit',)
('famille',)
('produit',)


In [6]:
# INSERTS INTO FAMILLE
# purge de la table
cur.execute('TRUNCATE TABLE famille')

# préparer les données
fam_df = fam_df.sort_values('Code article', ascending=False)
famindex = fam_df.reset_index()
famindex['Code article'] = range(1,31)
famindex.columns = ('nom', 'fam_id')

# création d’une table des inserts
inserts = []
for i, r in famindex.iterrows():
    inserts.append(f"({r['fam_id']}, '{r['nom']}')")

# création du megastring de la requête
sql = f'''
INSERT INTO famille (fam_id, nom)
VALUES
{', '.join(inserts)}
'''

cur.execute(sql)
db.commit()



In [7]:
# INSERTS INTO CONDITIONNEMENTS
# purge de la table
cur.execute('TRUNCATE TABLE condit')

# préparer les données
cond_df = cond_df.sort_values('Code article', ascending=False)
condindex = cond_df.reset_index()
condindex['Code article'] = range(1,condindex.shape[0]+1)
condindex.columns = ('nom', 'cond_id')

# création d’une table des inserts
inserts = []
for i, r in condindex.iterrows():
    inserts.append(f"({r['cond_id']}, '{r['nom']}')")

# création du megastring de la requête
sql = f'''
INSERT INTO condit (cond_id, nom)
VALUES
{', '.join(inserts)}
'''

cur.execute(sql)
db.commit()

In [8]:
# remplir la table principale

cur.execute('TRUNCATE TABLE produit')

# création de dictionnaires pour remplacer les noms de familles et conditions par leurs identifiants respectifs
fam_dic = famindex.to_dict('tight')
cond_dic = condindex.to_dict('tight')
repl_dict = {
    "FAMILLE ARTICLE": dict(fam_dic['data']),
    "CONDITION": dict(cond_dic['data'])
}
#boucle de remplacement
main_df = prod_df.copy()
for k,v in repl_dict.items() : {
    main_df[k].replace(v, inplace = True)
}
    
main_df.columns = ['code_art', 'libelle', 'fam_id', 'cond_id', 'prix']

inserts = []
for i, r in main_df.iterrows():
    tup = []
    tup.append(0)
    tup.append(r['fam_id'])
    tup.append(r['cond_id'])
    tup.append(r['code_art'])
    tup.append(r['libelle'])
    tup.append(float(r['prix'].replace(",",".")))
    tup = tuple(tup)
    inserts.append(tup)

sql = '''
INSERT INTO produit (prod_id, fam_id, cond_id, code_art, libelle, prix)
VALUES (%s, %s, %s, %s, %s, %s)
'''

cur.executemany(sql, inserts)
db.commit()


In [9]:
# editer les tables pour y spécifier l’existance de clef étrangères

# sql = '''
# ALTER TABLE produit
# DROP FOREIGN KEY produit_ibfk_1
# '''
# cur.execute(sql)

# sql = '''
# ALTER TABLE produit
# DROP FOREIGN KEY produit_ibfk_2
# '''
# cur.execute(sql)

sql = '''
ALTER TABLE produit
ADD CONSTRAINT FOREIGN KEY (fam_id)
REFERENCES famille(fam_id)
ON UPDATE CASCADE
ON DELETE CASCADE;
'''
cur.execute(sql)

sql = '''
ALTER TABLE produit
ADD CONSTRAINT FOREIGN KEY (cond_id)
REFERENCES condit(cond_id)
ON UPDATE CASCADE
ON DELETE CASCADE;
'''
cur.execute(sql)

cur.close()

True